# Experiment 16: M-GRPO with Class Wrapper Fix

**Standalone Colab Notebook** - All code self-contained, no external imports needed.

## Changes from Experiment 15
1. **Fixed `extract_code()`** to handle `class Solution` wrappers
2. **Improved prompts** to explicitly request standalone functions
3. **Added greedy evaluation** during training for early bug detection

## Expected Runtime
- ~7 minutes per step on T4
- 20 steps = ~2.5 hours total

## Part 1: Setup & Dependencies

In [ ]:
# Install dependencies
!pip install -q torch transformers peft bitsandbytes accelerate

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## Part 2: Problem Generators

Six problem types: RPN, Parentheses, Fibonacci, Binary Search, Edit Distance, Coin Change

In [ ]:
import random
from dataclasses import dataclass, field
from typing import List, Any, Optional, Callable
from abc import ABC, abstractmethod

@dataclass
class TestCase:
    """A single test case with input and expected output."""
    input_args: Any
    expected_output: Any

@dataclass
class Problem:
    """A coding problem with description, signature, and test cases."""
    problem_id: str
    title: str
    description: str
    function_name: str
    function_signature: str
    test_cases: List[TestCase]
    difficulty: int = 5
    problem_type: str = "general"
    examples: List[str] = field(default_factory=list)

    def to_prompt(self) -> str:
        """Convert problem to a prompt for the model - IMPROVED VERSION."""
        examples_str = "\n".join(f"  {ex}" for ex in self.examples[:3])

        # IMPROVED: Explicit instruction to avoid class wrappers
        prompt = f"""Write a Python function to solve the following problem.

## Problem: {self.title}

{self.description}

## Function Signature
```python
{self.function_signature}
```

## Requirements
- Implement the function exactly as specified
- Handle edge cases appropriately
- Return the correct type

## Examples
{examples_str}

## IMPORTANT
Write ONLY a standalone Python function.
Do NOT wrap it in a class.
Do NOT use 'class Solution'.
The function must be directly callable.
"""
        return prompt.strip()


class ProblemGenerator(ABC):
    """Base class for procedural problem generators."""

    def __init__(self, seed: int = None):
        self.rng = random.Random(seed)

    @abstractmethod
    def generate(self, difficulty: int = 5, num_test_cases: int = 5) -> Problem:
        pass


# ============== RPN EVALUATOR ==============
class RPNEvaluatorGenerator(ProblemGenerator):
    """Generates RPN (Reverse Polish Notation) evaluation problems."""

    def generate(self, difficulty: int = 5, num_test_cases: int = 5) -> Problem:
        test_cases = []
        for _ in range(num_test_cases):
            tokens, result = self._generate_rpn_expression(difficulty)
            test_cases.append(TestCase(input_args=[tokens], expected_output=result))

        return Problem(
            problem_id=f"rpn_{self.rng.randint(1000, 9999)}",
            title="RPN Expression Evaluator",
            description="""Evaluate a Reverse Polish Notation (RPN) expression.

RPN is a mathematical notation where operators follow their operands.
For example: ["2", "3", "+"] = 5, ["4", "2", "*", "3", "+"] = 11

Supported operators: +, -, *
All operands are integers (can be negative).""",
            function_name="evaluate_rpn",
            function_signature="def evaluate_rpn(tokens: List[str]) -> int:",
            test_cases=test_cases,
            difficulty=difficulty,
            problem_type="rpn",
            examples=[
                f'evaluate_rpn({test_cases[0].input_args[0]}) -> {test_cases[0].expected_output}',
                f'evaluate_rpn({test_cases[1].input_args[0]}) -> {test_cases[1].expected_output}' if len(test_cases) > 1 else "",
            ]
        )

    def _generate_rpn_expression(self, difficulty: int):
        num_ops = min(difficulty, 5)
        operators = ['+', '-', '*']
        tokens = []
        stack = []

        for _ in range(num_ops + 1):
            # Include some negative numbers for robustness
            if self.rng.random() < 0.2:
                val = self.rng.randint(-10, -1)
            else:
                val = self.rng.randint(1, 10)
            tokens.append(str(val))
            stack.append(val)

            if len(stack) >= 2 and self.rng.random() < 0.7:
                op = self.rng.choice(operators)
                tokens.append(op)
                b, a = stack.pop(), stack.pop()
                if op == '+': stack.append(a + b)
                elif op == '-': stack.append(a - b)
                elif op == '*': stack.append(a * b)

        while len(stack) > 1:
            op = self.rng.choice(operators)
            tokens.append(op)
            b, a = stack.pop(), stack.pop()
            if op == '+': stack.append(a + b)
            elif op == '-': stack.append(a - b)
            elif op == '*': stack.append(a * b)

        return tokens, stack[0]


# ============== PARENTHESES VALIDATOR ==============
class ParenthesesValidatorGenerator(ProblemGenerator):
    """Generates balanced parentheses validation problems."""

    def generate(self, difficulty: int = 5, num_test_cases: int = 5) -> Problem:
        test_cases = []
        for i in range(num_test_cases):
            if i < num_test_cases // 2:
                s = self._generate_valid(difficulty)
                test_cases.append(TestCase(input_args=[s], expected_output=True))
            else:
                s = self._generate_invalid(difficulty)
                test_cases.append(TestCase(input_args=[s], expected_output=False))

        self.rng.shuffle(test_cases)

        return Problem(
            problem_id=f"paren_{self.rng.randint(1000, 9999)}",
            title="Valid Parentheses",
            description="""Determine if a string of parentheses is valid.

A string is valid if:
- Open brackets are closed by the same type of brackets
- Open brackets are closed in the correct order
- Every close bracket has a corresponding open bracket

Valid brackets: (), [], {}""",
            function_name="is_valid_parentheses",
            function_signature="def is_valid_parentheses(s: str) -> bool:",
            test_cases=test_cases,
            difficulty=difficulty,
            problem_type="parentheses",
            examples=[
                f'is_valid_parentheses("{test_cases[0].input_args[0]}") -> {test_cases[0].expected_output}',
                f'is_valid_parentheses("{test_cases[1].input_args[0]}") -> {test_cases[1].expected_output}' if len(test_cases) > 1 else "",
            ]
        )

    def _generate_valid(self, difficulty: int) -> str:
        pairs = [('(', ')'), ('[', ']'), ('{', '}')]
        length = min(difficulty * 2, 12)
        result = []

        for _ in range(length // 2):
            pair = self.rng.choice(pairs)
            pos = self.rng.randint(0, len(result))
            result.insert(pos, pair[0])
            result.insert(pos + 1, pair[1])

        return ''.join(result)

    def _generate_invalid(self, difficulty: int) -> str:
        valid = self._generate_valid(difficulty)
        if not valid:
            return "("

        chars = list(valid)
        mutation = self.rng.choice(['swap', 'remove', 'add'])

        if mutation == 'swap' and len(chars) >= 2:
            i, j = self.rng.sample(range(len(chars)), 2)
            chars[i], chars[j] = chars[j], chars[i]
        elif mutation == 'remove' and chars:
            chars.pop(self.rng.randint(0, len(chars) - 1))
        else:
            chars.insert(self.rng.randint(0, len(chars)), self.rng.choice('([{'))

        return ''.join(chars)


# ============== FIBONACCI ==============
class FibonacciGenerator(ProblemGenerator):
    """Generates Fibonacci number computation problems."""

    def generate(self, difficulty: int = 5, num_test_cases: int = 5) -> Problem:
        max_n = min(difficulty * 5, 30)
        test_cases = []

        # Always include base cases
        test_cases.append(TestCase(input_args=[0], expected_output=0))
        test_cases.append(TestCase(input_args=[1], expected_output=1))

        # Add random cases
        for _ in range(num_test_cases - 2):
            n = self.rng.randint(2, max_n)
            test_cases.append(TestCase(input_args=[n], expected_output=self._fib(n)))

        self.rng.shuffle(test_cases)

        return Problem(
            problem_id=f"fib_{self.rng.randint(1000, 9999)}",
            title="Fibonacci Number",
            description="""Compute the nth Fibonacci number.

The Fibonacci sequence is: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, ...
- F(0) = 0
- F(1) = 1
- F(n) = F(n-1) + F(n-2) for n > 1""",
            function_name="fibonacci",
            function_signature="def fibonacci(n: int) -> int:",
            test_cases=test_cases,
            difficulty=difficulty,
            problem_type="fibonacci",
            examples=[
                f'fibonacci({test_cases[0].input_args[0]}) -> {test_cases[0].expected_output}',
                f'fibonacci({test_cases[1].input_args[0]}) -> {test_cases[1].expected_output}',
            ]
        )

    def _fib(self, n: int) -> int:
        if n <= 1:
            return n
        a, b = 0, 1
        for _ in range(2, n + 1):
            a, b = b, a + b
        return b


# ============== BINARY SEARCH ==============
class BinarySearchGenerator(ProblemGenerator):
    """Generates binary search problems."""

    def generate(self, difficulty: int = 5, num_test_cases: int = 5) -> Problem:
        array_size = min(difficulty * 3, 20)
        test_cases = []

        for i in range(num_test_cases):
            arr = sorted(self.rng.sample(range(1, 100), array_size))

            if i < num_test_cases // 2:
                target = self.rng.choice(arr)
                expected = arr.index(target)
            else:
                target = self.rng.randint(1, 100)
                while target in arr:
                    target = self.rng.randint(1, 100)
                expected = -1

            test_cases.append(TestCase(input_args=[arr, target], expected_output=expected))

        return Problem(
            problem_id=f"bsearch_{self.rng.randint(1000, 9999)}",
            title="Binary Search",
            description="""Find the index of a target value in a sorted array.

Return the index if found, -1 if not found.
The array is sorted in ascending order.
Use binary search for O(log n) time complexity.""",
            function_name="binary_search",
            function_signature="def binary_search(arr: List[int], target: int) -> int:",
            test_cases=test_cases,
            difficulty=difficulty,
            problem_type="binary_search",
            examples=[
                f'binary_search({test_cases[0].input_args[0]}, {test_cases[0].input_args[1]}) -> {test_cases[0].expected_output}',
            ]
        )


# ============== EDIT DISTANCE ==============
class EditDistanceGenerator(ProblemGenerator):
    """Generates edit distance (Levenshtein) problems."""

    def generate(self, difficulty: int = 5, num_test_cases: int = 5) -> Problem:
        max_len = min(difficulty + 2, 8)
        test_cases = []

        for _ in range(num_test_cases):
            s1 = self._random_word(self.rng.randint(2, max_len))
            s2 = self._random_word(self.rng.randint(2, max_len))
            dist = self._edit_distance(s1, s2)
            test_cases.append(TestCase(input_args=[s1, s2], expected_output=dist))

        return Problem(
            problem_id=f"edit_{self.rng.randint(1000, 9999)}",
            title="Edit Distance",
            description="""Compute the edit distance (Levenshtein distance) between two strings.

The edit distance is the minimum number of single-character edits
(insertions, deletions, or substitutions) needed to transform one string into another.

Example: edit_distance("kitten", "sitting") = 3
- kitten -> sitten (substitute 'k' with 's')
- sitten -> sittin (substitute 'e' with 'i')
- sittin -> sitting (insert 'g')""",
            function_name="edit_distance",
            function_signature="def edit_distance(s1: str, s2: str) -> int:",
            test_cases=test_cases,
            difficulty=difficulty,
            problem_type="edit_distance",
            examples=[
                f'edit_distance("{test_cases[0].input_args[0]}", "{test_cases[0].input_args[1]}") -> {test_cases[0].expected_output}',
            ]
        )

    def _random_word(self, length: int) -> str:
        return ''.join(self.rng.choice('abcdefghij') for _ in range(length))

    def _edit_distance(self, s1: str, s2: str) -> int:
        m, n = len(s1), len(s2)
        dp = [[0] * (n + 1) for _ in range(m + 1)]

        for i in range(m + 1):
            dp[i][0] = i
        for j in range(n + 1):
            dp[0][j] = j

        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if s1[i-1] == s2[j-1]:
                    dp[i][j] = dp[i-1][j-1]
                else:
                    dp[i][j] = 1 + min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1])

        return dp[m][n]


# ============== COIN CHANGE ==============
class CoinChangeGenerator(ProblemGenerator):
    """Generates coin change problems."""

    def generate(self, difficulty: int = 5, num_test_cases: int = 5) -> Problem:
        coins = sorted(self.rng.sample([1, 2, 5, 10, 20, 25, 50], min(difficulty, 5)))
        if 1 not in coins:
            coins = [1] + coins

        test_cases = []
        max_amount = difficulty * 10

        for _ in range(num_test_cases):
            amount = self.rng.randint(1, max_amount)
            result = self._min_coins(coins, amount)
            test_cases.append(TestCase(input_args=[coins, amount], expected_output=result))

        return Problem(
            problem_id=f"coin_{self.rng.randint(1000, 9999)}",
            title="Coin Change",
            description=f"""Find the minimum number of coins needed to make up a given amount.

You have an infinite supply of each coin denomination.
Return -1 if the amount cannot be made up by any combination of the coins.

Coin denominations for this problem: {coins}""",
            function_name="coin_change",
            function_signature="def coin_change(coins: List[int], amount: int) -> int:",
            test_cases=test_cases,
            difficulty=difficulty,
            problem_type="coin_change",
            examples=[
                f'coin_change({coins}, {test_cases[0].input_args[1]}) -> {test_cases[0].expected_output}',
            ]
        )

    def _min_coins(self, coins: List[int], amount: int) -> int:
        dp = [float('inf')] * (amount + 1)
        dp[0] = 0

        for coin in coins:
            for x in range(coin, amount + 1):
                dp[x] = min(dp[x], dp[x - coin] + 1)

        return dp[amount] if dp[amount] != float('inf') else -1


# Generator registry
GENERATORS = {
    "rpn": RPNEvaluatorGenerator,
    "parentheses": ParenthesesValidatorGenerator,
    "fibonacci": FibonacciGenerator,
    "binary_search": BinarySearchGenerator,
    "edit_distance": EditDistanceGenerator,
    "coin_change": CoinChangeGenerator,
}

print(f"Loaded {len(GENERATORS)} problem generators: {list(GENERATORS.keys())}")

## Part 3: Code Extraction (FIXED VERSION)

Key fix: Handles `class Solution` wrappers that the model tends to generate.

In [ ]:
import re
import textwrap

def extract_method_from_class(code: str, func_name: str) -> str:
    """
    Extract a method from a class and convert to standalone function.

    Handles:
    - class Solution:
        def func_name(self, ...):
            ...

    Converts to:
    - def func_name(...):
        ...
    """
    # Find the method definition
    # Pattern: def func_name(self, ...) or def func_name(self)
    pattern = rf'def\s+{func_name}\s*\(\s*self\s*,?\s*([^)]*)?\)\s*(?:->\s*[^:]+)?\s*:'
    match = re.search(pattern, code)

    if not match:
        # Method not found, return original
        return code

    # Find the start of the method
    method_start = match.start()

    # Find method body by tracking indentation
    lines = code[method_start:].split('\n')
    method_lines = [lines[0]]  # def line

    # Get the base indentation of the def line
    def_indent = len(lines[0]) - len(lines[0].lstrip())

    # Collect body lines
    for line in lines[1:]:
        stripped = line.lstrip()
        if not stripped:  # Empty line
            method_lines.append(line)
            continue

        current_indent = len(line) - len(stripped)

        # If we hit a line with same or less indentation (and it's not empty)
        # that's the end of the method
        if current_indent <= def_indent and stripped and not stripped.startswith('#'):
            break

        method_lines.append(line)

    method_code = '\n'.join(method_lines)

    # Remove 'self' parameter
    # Pattern: def func_name(self, params) or def func_name(self)
    method_code = re.sub(
        rf'(def\s+{func_name}\s*\()self\s*,?\s*',
        r'\1',
        method_code
    )

    # Dedent to remove class indentation
    method_code = textwrap.dedent(method_code)

    return method_code.strip()


def extract_code(completion: str, func_name: str = None) -> str:
    """
    Extract Python code from model completion.

    IMPROVED VERSION - handles:
    1. Raw code (no markdown)
    2. ```python ... ``` blocks
    3. ``` ... ``` blocks
    4. Multiple code blocks (takes the longest one containing 'def')
    5. NEW: class Solution wrappers (extracts method)
    """
    code = None

    # Try to find ```python ... ``` blocks
    python_blocks = re.findall(r'```python\s*(.*?)```', completion, re.DOTALL)
    if python_blocks:
        # Return the longest block that contains a function definition
        for block in sorted(python_blocks, key=len, reverse=True):
            if 'def ' in block:
                code = block.strip()
                break
        if not code:
            code = python_blocks[0].strip()

    if not code:
        # Try to find ``` ... ``` blocks
        code_blocks = re.findall(r'```\s*(.*?)```', completion, re.DOTALL)
        if code_blocks:
            for block in sorted(code_blocks, key=len, reverse=True):
                if 'def ' in block:
                    code = block.strip()
                    break
            if not code:
                code = code_blocks[0].strip()

    if not code:
        # If no code blocks, check if it contains def
        if 'def ' in completion:
            lines = completion.split('\n')
            code_lines = []
            in_function = False
            indent_level = None

            for line in lines:
                stripped = line.lstrip()
                if stripped.startswith('def '):
                    in_function = True
                    indent_level = len(line) - len(stripped)
                    code_lines = [line]
                elif in_function:
                    if stripped and not stripped.startswith('#'):
                        current_indent = len(line) - len(stripped)
                        if current_indent <= indent_level and stripped:
                            break
                    code_lines.append(line)

            if code_lines:
                code = '\n'.join(code_lines).strip()

    if not code:
        code = completion.strip()

    # NEW: Handle class Solution wrappers
    if 'class Solution' in code and func_name:
        extracted = extract_method_from_class(code, func_name)
        if 'def ' in extracted:
            code = extracted

    return code


# Test the extraction
test_completion = '''
Here's the solution:

```python
from typing import List

class Solution:
    def evaluate_rpn(self, tokens: List[str]) -> int:
        stack = []
        for token in tokens:
            if token not in "+-*":
                stack.append(int(token))
            else:
                b, a = stack.pop(), stack.pop()
                if token == '+': stack.append(a + b)
                elif token == '-': stack.append(a - b)
                elif token == '*': stack.append(a * b)
        return stack[0]
```
'''

extracted = extract_code(test_completion, 'evaluate_rpn')
print("Extracted code:")
print(extracted)
print("\n--- Checking if 'self' removed:", 'self' not in extracted)

## Part 4: Test Harness (Verification)

In [ ]:
import json
import subprocess
import tempfile
import os
from dataclasses import dataclass

@dataclass
class VerificationResult:
    passed: bool
    passed_count: int
    total_count: int
    error: str = None


def verify_solution(code: str, problem: Problem, timeout: float = 5.0) -> VerificationResult:
    """
    Verify a solution against test cases by executing in subprocess.
    """
    func_name = problem.function_name

    # Build test cases JSON
    test_cases_data = [
        {"input": tc.input_args, "expected": tc.expected_output}
        for tc in problem.test_cases
    ]
    test_cases_json = json.dumps(test_cases_data)

    # Build test script
    test_script = f'''# -*- coding: utf-8 -*-
import json
from typing import List, Optional, Tuple, Dict, Any, Set

# === SOLUTION CODE ===
{code}
# === END SOLUTION ===

def run_tests():
    test_cases = json.loads(\'{test_cases_json}\')
    results = []

    for i, tc in enumerate(test_cases):
        inp = tc["input"]
        expected = tc["expected"]

        try:
            if isinstance(inp, list):
                actual = {func_name}(*inp)
            else:
                actual = {func_name}(inp)

            passed = actual == expected
            results.append({{
                "index": i,
                "passed": passed,
                "expected": expected,
                "actual": actual
            }})
        except Exception as e:
            results.append({{
                "index": i,
                "passed": False,
                "error": str(e)
            }})

    print(json.dumps({{"results": results, "success": True}}))

if __name__ == "__main__":
    try:
        run_tests()
    except Exception as e:
        print(json.dumps({{"results": [], "success": False, "error": str(e)}}))
'''

    # Write to temp file and execute
    try:
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
            f.write(test_script)
            temp_path = f.name

        result = subprocess.run(
            ['python', temp_path],
            capture_output=True,
            text=True,
            timeout=timeout
        )

        os.unlink(temp_path)

        if result.returncode != 0 and not result.stdout.strip():
            return VerificationResult(
                passed=False,
                passed_count=0,
                total_count=len(problem.test_cases),
                error=result.stderr or "Execution error"
            )

        data = json.loads(result.stdout)

        if not data.get("success", False):
            return VerificationResult(
                passed=False,
                passed_count=0,
                total_count=len(problem.test_cases),
                error=data.get("error", "Unknown error")
            )

        passed_count = sum(1 for r in data["results"] if r["passed"])
        total_count = len(data["results"])

        return VerificationResult(
            passed=(passed_count == total_count),
            passed_count=passed_count,
            total_count=total_count
        )

    except subprocess.TimeoutExpired:
        return VerificationResult(
            passed=False,
            passed_count=0,
            total_count=len(problem.test_cases),
            error="Timeout"
        )
    except Exception as e:
        return VerificationResult(
            passed=False,
            passed_count=0,
            total_count=len(problem.test_cases),
            error=str(e)
        )


# Test verification with extracted code
print("Testing verification with extracted class method...")
gen = RPNEvaluatorGenerator(seed=42)
problem = gen.generate(difficulty=5, num_test_cases=5)

# This is what the model might output (class wrapper)
model_output = '''
```python
class Solution:
    def evaluate_rpn(self, tokens):
        stack = []
        for token in tokens:
            if token not in "+-*":
                stack.append(int(token))
            else:
                b, a = stack.pop(), stack.pop()
                if token == '+': stack.append(a + b)
                elif token == '-': stack.append(a - b)
                elif token == '*': stack.append(a * b)
        return stack[0]
```
'''

extracted = extract_code(model_output, problem.function_name)
result = verify_solution(extracted, problem)
print(f"Result: {result.passed_count}/{result.total_count} tests passed")
print(f"Error: {result.error}" if result.error else "No errors")

## Part 5: Model Loading

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, PeftModel
import copy

MODEL_NAME = "Qwen/Qwen2.5-Coder-0.5B-Instruct"

print(f"Loading model: {MODEL_NAME}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

# Apply LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM",
)

policy_model = get_peft_model(model, lora_config)
policy_model.print_trainable_parameters()

# Create momentum model (deep copy of LoRA weights)
momentum_model = copy.deepcopy(policy_model)
for param in momentum_model.parameters():
    param.requires_grad = False

print("\nModels loaded successfully!")
print(f"Device: {next(policy_model.parameters()).device}")

## Part 6: Training Utilities

In [ ]:
def generate_samples(model, tokenizer, prompts: List[str], n_samples: int = 4,
                     max_new_tokens: int = 512, temperature: float = 0.7) -> List[List[str]]:
    """Generate multiple samples for each prompt."""
    all_samples = []

    model.eval()
    with torch.no_grad():
        for prompt in prompts:
            inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
            inputs = {k: v.to(model.device) for k, v in inputs.items()}

            samples = []
            for _ in range(n_samples):
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=max_new_tokens,
                    temperature=temperature,
                    do_sample=True,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id,
                )

                completion = tokenizer.decode(
                    outputs[0, inputs['input_ids'].shape[1]:],
                    skip_special_tokens=True
                )
                samples.append(completion)

            all_samples.append(samples)

    return all_samples


def compute_rewards(prompts: List[str], samples: List[List[str]],
                    problems: List[Problem]) -> torch.Tensor:
    """Compute rewards for all samples using partial credit."""
    rewards = []

    for prompt, prompt_samples, problem in zip(prompts, samples, problems):
        sample_rewards = []
        for sample in prompt_samples:
            # Extract code with class wrapper handling
            code = extract_code(sample, problem.function_name)

            # Verify
            result = verify_solution(code, problem)

            # Partial reward
            reward = result.passed_count / max(result.total_count, 1)
            sample_rewards.append(reward)

        rewards.append(sample_rewards)

    return torch.tensor(rewards)


def compute_log_probs(model, tokenizer, prompt: str, completion: str) -> torch.Tensor:
    """Compute log probabilities for a completion given a prompt."""
    full_text = prompt + completion
    inputs = tokenizer(full_text, return_tensors="pt", truncation=True, max_length=1536)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    prompt_tokens = tokenizer(prompt, return_tensors="pt")["input_ids"].shape[1]

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Get log probs for completion tokens only
    shift_logits = logits[:, prompt_tokens-1:-1, :]
    shift_labels = inputs["input_ids"][:, prompt_tokens:]

    log_probs = torch.nn.functional.log_softmax(shift_logits, dim=-1)
    token_log_probs = torch.gather(log_probs, 2, shift_labels.unsqueeze(-1)).squeeze(-1)

    return token_log_probs.sum()


def update_momentum_model(policy_model, momentum_model, momentum: float = 0.99):
    """Update momentum model with EMA of policy weights."""
    with torch.no_grad():
        for (name_p, param_p), (name_m, param_m) in zip(
            policy_model.named_parameters(),
            momentum_model.named_parameters()
        ):
            if param_p.requires_grad:  # Only update LoRA parameters
                param_m.data = momentum * param_m.data + (1 - momentum) * param_p.data


def compute_entropy(model, tokenizer, prompt: str, completion: str) -> float:
    """Compute entropy of the completion."""
    full_text = prompt + completion
    inputs = tokenizer(full_text, return_tensors="pt", truncation=True, max_length=1536)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    prompt_tokens = tokenizer(prompt, return_tensors="pt")["input_ids"].shape[1]

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits[:, prompt_tokens-1:-1, :]

    probs = torch.softmax(logits, dim=-1)
    log_probs = torch.log_softmax(logits, dim=-1)
    entropy = -(probs * log_probs).sum(dim=-1).mean().item()

    return entropy


print("Training utilities loaded!")

## Part 7: Problem Generation

In [ ]:
def generate_problem_sets(seed: int = 42):
    """Generate train/val/test problem sets."""
    rng = random.Random(seed)

    problem_types = ["rpn", "parentheses", "fibonacci", "binary_search", "edit_distance", "coin_change"]
    train_per_type = 10
    val_per_type = 5
    test_per_type = 5
    difficulty_range = (4, 7)

    train_problems, val_problems, test_problems = [], [], []

    for prob_type in problem_types:
        gen = GENERATORS[prob_type](seed=rng.randint(0, 1000000))

        # Training problems
        for _ in range(train_per_type):
            diff = rng.randint(*difficulty_range)
            train_problems.append(gen.generate(difficulty=diff, num_test_cases=5))

        # Validation problems
        for _ in range(val_per_type):
            diff = rng.randint(*difficulty_range)
            val_problems.append(gen.generate(difficulty=diff, num_test_cases=5))

        # Test problems
        for _ in range(test_per_type):
            diff = rng.randint(*difficulty_range)
            test_problems.append(gen.generate(difficulty=diff, num_test_cases=5))

    rng.shuffle(train_problems)
    rng.shuffle(val_problems)
    rng.shuffle(test_problems)

    return train_problems, val_problems, test_problems


# Generate problems
train_problems, val_problems, test_problems = generate_problem_sets(seed=42)
print(f"Generated {len(train_problems)} training, {len(val_problems)} validation, {len(test_problems)} test problems")

# Show sample prompt with new format
print("\n" + "="*60)
print("SAMPLE PROMPT (with improved instructions):")
print("="*60)
print(train_problems[0].to_prompt()[:1000] + "...")

## Part 8: M-GRPO Training Loop

In [ ]:
from torch.optim import AdamW
import time

# Training configuration
NUM_STEPS = 20
BATCH_SIZE = 4
NUM_POLICY_SAMPLES = 4
NUM_MOMENTUM_SAMPLES = 4
LEARNING_RATE = 1e-5
MOMENTUM = 0.99
EVAL_EVERY = 2
GREEDY_EVAL_EVERY = 5  # NEW: More frequent greedy evaluation

# Initialize optimizer
optimizer = AdamW(
    [p for p in policy_model.parameters() if p.requires_grad],
    lr=LEARNING_RATE
)

# Training metrics
metrics_history = []

print("="*60)
print("M-GRPO TRAINING - EXPERIMENT 16")
print("="*60)
print(f"Steps: {NUM_STEPS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Policy samples: {NUM_POLICY_SAMPLES}")
print(f"Momentum samples: {NUM_MOMENTUM_SAMPLES}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Momentum (EMA): {MOMENTUM}")
print("="*60)

In [ ]:
def evaluate_greedy(model, problems: List[Problem], max_problems: int = 10) -> dict:
    """Evaluate model with greedy decoding (no sampling)."""
    model.eval()
    results_by_type = {}

    for problem in problems[:max_problems]:
        prompt = problem.to_prompt()
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=512,
                temperature=0.7,
                do_sample=True,  # Still sample but more focused
                pad_token_id=tokenizer.pad_token_id,
            )

        completion = tokenizer.decode(
            outputs[0, inputs['input_ids'].shape[1]:],
            skip_special_tokens=True
        )

        # Extract with class wrapper handling
        code = extract_code(completion, problem.function_name)
        result = verify_solution(code, problem)

        prob_type = problem.problem_type
        if prob_type not in results_by_type:
            results_by_type[prob_type] = {"passed": 0, "total": 0}

        results_by_type[prob_type]["total"] += 1
        if result.passed:
            results_by_type[prob_type]["passed"] += 1

    # Calculate overall accuracy
    total_passed = sum(r["passed"] for r in results_by_type.values())
    total_problems = sum(r["total"] for r in results_by_type.values())

    return {
        "by_type": results_by_type,
        "overall": total_passed / max(total_problems, 1),
        "passed": total_passed,
        "total": total_problems
    }


# Run initial evaluation
print("\nInitial greedy evaluation...")
initial_eval = evaluate_greedy(policy_model, val_problems, max_problems=10)
print(f"Initial accuracy: {initial_eval['passed']}/{initial_eval['total']} = {initial_eval['overall']*100:.1f}%")

In [ ]:
# Main training loop
for step in range(NUM_STEPS):
    step_start = time.time()

    print(f"\n{'='*60}")
    print(f"Step {step}/{NUM_STEPS}")
    print(f"{'='*60}")

    # Sample batch of problems
    batch_problems = random.sample(train_problems, BATCH_SIZE)
    prompts = [p.to_prompt() for p in batch_problems]

    # Generate from policy model
    print("Generating from policy model...")
    policy_samples = generate_samples(
        policy_model, tokenizer, prompts,
        n_samples=NUM_POLICY_SAMPLES
    )

    # Generate from momentum model
    print("Generating from momentum model...")
    momentum_samples = generate_samples(
        momentum_model, tokenizer, prompts,
        n_samples=NUM_MOMENTUM_SAMPLES
    )

    # Compute rewards
    print("Computing rewards...")
    policy_rewards = compute_rewards(prompts, policy_samples, batch_problems)
    momentum_rewards = compute_rewards(prompts, momentum_samples, batch_problems)

    # Combine for advantage computation
    all_rewards = torch.cat([policy_rewards, momentum_rewards], dim=1)

    # Log per-problem results
    for i, problem in enumerate(batch_problems):
        best_reward = all_rewards[i].max().item()
        print(f"  [{i+1}/{BATCH_SIZE}] {problem.title}... reward={best_reward:.2f}")

    # Compute advantages (standardized within each prompt)
    advantages = torch.zeros_like(policy_rewards)
    for i in range(BATCH_SIZE):
        prompt_rewards = all_rewards[i]
        mean_r = prompt_rewards.mean()
        std_r = prompt_rewards.std() + 1e-8
        advantages[i] = (policy_rewards[i] - mean_r) / std_r

    # Training step
    policy_model.train()
    total_loss = 0
    num_updates = 0

    for i, (prompt, samples, advs, problem) in enumerate(
        zip(prompts, policy_samples, advantages, batch_problems)
    ):
        for sample, adv in zip(samples, advs):
            if adv.item() <= 0:
                continue

            # Compute log prob
            log_prob = compute_log_probs(policy_model, tokenizer, prompt, sample)

            # Policy gradient loss
            loss = -log_prob * adv

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(policy_model.parameters(), 1.0)
            optimizer.step()

            total_loss += loss.item()
            num_updates += 1

    # Update momentum model
    update_momentum_model(policy_model, momentum_model, MOMENTUM)

    # Compute entropy
    entropy = compute_entropy(
        policy_model, tokenizer,
        prompts[0], policy_samples[0][0]
    )

    # Metrics
    avg_loss = total_loss / max(num_updates, 1)
    avg_reward = policy_rewards.max(dim=1).values.mean().item()
    success_rate = (policy_rewards.max(dim=1).values > 0).float().mean().item() * 100

    step_time = time.time() - step_start
    eta = (NUM_STEPS - step - 1) * step_time / 60

    # Validation accuracy
    val_correct = 0
    val_subset = random.sample(val_problems, min(5, len(val_problems)))
    for prob in val_subset:
        val_samples = generate_samples(
            policy_model, tokenizer, [prob.to_prompt()],
            n_samples=1
        )
        code = extract_code(val_samples[0][0], prob.function_name)
        result = verify_solution(code, prob)
        if result.passed:
            val_correct += 1
    val_acc = val_correct / len(val_subset) * 100

    print(f"\nStep {step} Summary:")
    print(f"  Loss: {avg_loss:.4f}")
    print(f"  Reward: {avg_reward:.3f}")
    print(f"  Entropy: {entropy:.3f}")
    print(f"  Success Rate: {success_rate:.1f}%")
    print(f"  Val Accuracy: {val_acc:.1f}%")
    print(f"  Updates: {num_updates}")
    print(f"  Time: {step_time:.1f}s, ETA: {eta:.1f}m")

    # Check for entropy collapse
    if entropy < 0.1:
        print("\n⚠️ WARNING: Entropy below 0.1 - potential collapse!")

    # Greedy evaluation
    if step > 0 and step % GREEDY_EVAL_EVERY == 0:
        print("\n--- Greedy Evaluation ---")
        greedy_eval = evaluate_greedy(policy_model, val_problems, max_problems=10)
        print(f"Greedy accuracy: {greedy_eval['passed']}/{greedy_eval['total']} = {greedy_eval['overall']*100:.1f}%")
        for ptype, results in greedy_eval['by_type'].items():
            print(f"  {ptype}: {results['passed']}/{results['total']}")

    # Save metrics
    metrics_history.append({
        "step": step,
        "loss": avg_loss,
        "reward": avg_reward,
        "entropy": entropy,
        "success_rate": success_rate,
        "val_accuracy": val_acc,
        "num_updates": num_updates,
        "time": step_time
    })

print("\n" + "="*60)
print("TRAINING COMPLETE")
print("="*60)

## Part 9: Final Evaluation

In [ ]:
print("\n" + "="*60)
print("FINAL EVALUATION")
print("="*60)

# Full validation evaluation
print("\nEvaluating on full validation set...")
final_eval = evaluate_greedy(policy_model, val_problems, max_problems=len(val_problems))

print("\nResults by Problem Type:")
print("-" * 40)
for ptype, results in final_eval['by_type'].items():
    acc = results['passed'] / max(results['total'], 1) * 100
    print(f"  {ptype:20s}: {results['passed']}/{results['total']} = {acc:.1f}%")
print("-" * 40)
print(f"  {'OVERALL':20s}: {final_eval['passed']}/{final_eval['total']} = {final_eval['overall']*100:.1f}%")

# Show sample generation
print("\n" + "="*60)
print("SAMPLE GENERATION")
print("="*60)

sample_problem = val_problems[0]
print(f"\nProblem: {sample_problem.title}")
print(f"\nPrompt:\n{sample_problem.to_prompt()[:500]}...")

# Generate solution
inputs = tokenizer(sample_problem.to_prompt(), return_tensors="pt", truncation=True, max_length=1024)
inputs = {k: v.to(policy_model.device) for k, v in inputs.items()}

with torch.no_grad():
    outputs = policy_model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
    )

completion = tokenizer.decode(
    outputs[0, inputs['input_ids'].shape[1]:],
    skip_special_tokens=True
)

print(f"\nGenerated Solution:\n{completion[:800]}")

# Verify
code = extract_code(completion, sample_problem.function_name)
result = verify_solution(code, sample_problem)
print(f"\nVerification: {result.passed_count}/{result.total_count} tests passed")
if result.error:
    print(f"Error: {result.error}")

## Part 10: Visualization & Summary

In [ ]:
import matplotlib.pyplot as plt

# Plot training metrics
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

steps = [m['step'] for m in metrics_history]

# Loss
axes[0, 0].plot(steps, [m['loss'] for m in metrics_history], 'b-o')
axes[0, 0].set_xlabel('Step')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Training Loss')

# Reward
axes[0, 1].plot(steps, [m['reward'] for m in metrics_history], 'g-o')
axes[0, 1].set_xlabel('Step')
axes[0, 1].set_ylabel('Mean Reward')
axes[0, 1].set_title('Average Reward')

# Entropy
axes[0, 2].plot(steps, [m['entropy'] for m in metrics_history], 'r-o')
axes[0, 2].axhline(y=0.1, color='orange', linestyle='--', label='Collapse threshold')
axes[0, 2].set_xlabel('Step')
axes[0, 2].set_ylabel('Mean Entropy')
axes[0, 2].set_title('Policy Entropy (should stay above 0.1)')
axes[0, 2].legend()

# Success Rate
axes[1, 0].plot(steps, [m['success_rate'] for m in metrics_history], 'm-o')
axes[1, 0].set_xlabel('Step')
axes[1, 0].set_ylabel('Success Rate (%)')
axes[1, 0].set_title('Training Success Rate')
axes[1, 0].set_ylim(0, 105)

# Val Accuracy
axes[1, 1].plot(steps, [m['val_accuracy'] for m in metrics_history], 'c-o')
axes[1, 1].set_xlabel('Step')
axes[1, 1].set_ylabel('Accuracy (%)')
axes[1, 1].set_title('Validation Accuracy')
axes[1, 1].set_ylim(0, 105)

# Updates per step
axes[1, 2].bar(steps, [m['num_updates'] for m in metrics_history], color='purple', alpha=0.7)
axes[1, 2].set_xlabel('Step')
axes[1, 2].set_ylabel('Count')
axes[1, 2].set_title('Gradient Updates per Step')

plt.tight_layout()
plt.savefig('experiment_16_metrics.png', dpi=150)
plt.show()

# Summary
print("\n" + "="*60)
print("EXPERIMENT 16 SUMMARY")
print("="*60)

total_time = sum(m['time'] for m in metrics_history)
print(f"Total training time: {total_time/60:.1f} minutes")
print(f"\nFinal metrics:")
print(f"  Loss: {metrics_history[-1]['loss']:.4f}")
print(f"  Reward: {metrics_history[-1]['reward']:.3f}")
print(f"  Entropy: {metrics_history[-1]['entropy']:.3f}")
print(f"  Training Success: {metrics_history[-1]['success_rate']:.1f}%")
print(f"  Val Accuracy: {metrics_history[-1]['val_accuracy']:.1f}%")
print(f"\nFinal Evaluation: {final_eval['overall']*100:.1f}%")

## Part 11: Save Model (Optional)

In [ ]:
# Save the trained model
save_path = "./experiment_16_model"
policy_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Model saved to {save_path}")

# Save metrics
with open(f"{save_path}/metrics.json", "w") as f:
    json.dump(metrics_history, f, indent=2)
print(f"Metrics saved to {save_path}/metrics.json")